In [1]:
from IPython import get_ipython
from IPython.display import clear_output, Image, display, HTML

# Exploring the Convection-Diffusion Equation
## Using the Finite Element Formulation


In this Notebook we explore the steady convection-diffusion-reaction equation:

$$
\begin{aligned}
\mathbf{\beta} \cdot \nabla \phi  - \epsilon \Delta \phi = f 
\end{aligned}
$$



The Peclet number is the ratio between the advective transport and the diffusive transport rates. The Peclet number can be expressed as the global Peclet number, respective to the whole domain, and as the local Peclet number, related to a given element of a finite element mesh. Those are defined as:
$$
\begin{aligned}
Pe &= \dfrac{||\mathbf{\beta}||}{2\epsilon}\\
Pe_{mesh} &= \dfrac{||\mathbf{\beta}||h}{2\epsilon}
\end{aligned}
$$


Problems with large $Pe$, that is, advection dominated, reveal numerical difficulties. 

<img src="Pe.png" align="center"/>

The Galerkin Method presents unstable behavior as $Pe_{mesh}$ increases.

## Weak formulation

We consider a square unit domain $\Omega$ with Dirichlet boundary conditions on $\partial \Omega$. The discretization of the domain $\Omega$ into $nel$ number of elements is such that $\Omega=\cup^{nel}_{e=1} \Omega^e$ and $\emptyset =\cap^{nel}_{e=1}\Omega^e$. Considering ${S}^h$ the space of the test functions and ${V}^h$ the space of the weight functions, the weak (variational) formulation of the CDR equation becomes: find $\phi^h(\textbf{x}) \in {S}^h$ such that $\forall w^h \in {V}^h$: 

$$
\begin{aligned}
    \int_{\Omega}\bigg(w^h(\mathbf{\beta}\cdot\nabla\phi^h) + w^h \nabla \cdot (\epsilon\nabla\phi^h)\bigg)d\Omega = \int_{\Omega}w^hfd\Omega \hspace{1cm}\forall w^h \in {V}^h
\end{aligned}
$$

Using the divergence theorem and the fact that $w^h = 0$ on $\partial \Omega$, the previous equation becomes: 
$$
\begin{aligned}
    \int_{\Omega}\bigg(w^h(\mathbf{\beta}\cdot\nabla\phi^h) + \nabla w^h  \cdot (\epsilon\nabla\phi^h) \bigg)d\Omega = \int_{\Omega}w^hfd\Omega +  \int_{\partial\Omega_N}w^hgd\Gamma \hspace{1cm}\forall w^h \in {V}^h
\end{aligned}
$$

In [2]:
get_ipython().run_line_magic('matplotlib', 'inline')
from ipywidgets import interact, interactive
from IPython.display import clear_output, display, HTML

In [3]:
from fenics import *
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import time
import warnings

warnings.filterwarnings("ignore")

class Boundaries:
    def left(x, on_boundary):
        return x[0] < DOLFIN_EPS and on_boundary
    def top(x, on_boundary):
        return x[1] > 1.0 - DOLFIN_EPS and on_boundary
    def right(x, on_boundary):
        return x[0] > 1.0 - DOLFIN_EPS and on_boundary
    def bottom(x, on_boundary):
        return x[1] < DOLFIN_EPS and on_boundary


def mesh2triang(mesh):
    xy = mesh.coordinates()
    return tri.Triangulation(xy[:, 0], xy[:, 1], mesh.cells())


def myplot(mymesh, field, N):
    #plt.gca().set_aspect('equal')
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
    ax1.triplot(mesh2triang(mymesh), color='k')
    C = field.compute_vertex_values(mymesh)
    X, Y = mymesh.coordinates()[:,0], mymesh.coordinates()[:,1]
    Cr   = C.reshape(N,N)
    plt.yticks(ticks = [])
    plt.xticks(ticks = [])
    ax2.contourf(Cr)
    mappable =  matplotlib.cm.ScalarMappable(norm=None, cmap=None)
    mappable.set_array(C)
    fig.colorbar(mappable)



def cdr2d(N , Pe , SUPG ):
    #================= Problem Definition ===================
    mesh        = UnitSquareMesh(N-1, N-1)
    h           = CellDiameter(mesh)
    Q           = FunctionSpace(mesh, "CG", 1)
    dphi, v     = TrialFunction(Q), TestFunction(Q)
    phi         = Function(Q)
    V1          = VectorElement("Lagrange", mesh.ufl_cell(), 1)
    V           = FunctionSpace(mesh, V1)
    velocity    = Function(V)
    f           = Constant(1.0)
    
    #================= Parameters ===========================
    if Pe == 0:
        k           = Constant(1.0)
        velocity    = Constant((0.0, 0.0))
    else:
        eps         = np.sqrt(2.)/(2*Pe)
        k           = Constant(eps)
        velocity    = as_vector([1.0, 1.0])

    #================= Boundary Conditions===================
    BC1         = Constant(0.0)
    bc_left     = DirichletBC(Q, BC1, Boundaries.left)
    bc_top      = DirichletBC(Q, BC1, Boundaries.top)
    bc_right    = DirichletBC(Q, BC1, Boundaries.right)
    bc_bottom   = DirichletBC(Q, BC1, Boundaries.bottom)
    bcs         = [bc_left, bc_top, bc_right, bc_bottom]

    #================= Variational Form ====================
    F = v*dot(velocity, grad(dphi))*dx + k*dot(grad(v), grad(dphi))*dx - f*v*dx
    
    if SUPG == True:
        r = dot(velocity, grad(dphi)) - k*div(grad(dphi)) - f
        vnorm = sqrt(dot(velocity, velocity))
        delta = h/(2.0*vnorm)
        F += delta*dot(velocity, grad(v))*r*dx
 


    #====================== Solver =========================
    a = lhs(F)
    L = rhs(F)
    solve(a == L, phi, bcs) 
    d = np.sqrt(2*(1./N))
    print("Local Pe: ", Pe*d)

    
    myplot(mesh, phi, N)


    return 0


In [4]:
import ipywidgets as widgets
SUPG = widgets.Checkbox(
    value=False,
    description="SUPG stabilization",
    disabled=False,
    indent=True
)
Pe   = widgets.FloatSlider(
    value=0,
    min=0,
    step=1.,
    max=100.,
    description= "Pe"
)
N    = widgets.IntSlider(
    value=3,
    min=3,
    max=50,
    step=1,
    description="Nodes / dim"
)

w = interactive(cdr2d, N=N, Pe=Pe, SUPG=SUPG)
display(w)



interactive(children=(IntSlider(value=3, description='Nodes / dim', max=50, min=3), FloatSlider(value=0.0, des…